In [1]:
import pandas
import numpy as np
from Recommender.DeepContent.Recommender import DeepContentRecommender
from Recommender.DeepContent.LabellingModel import SongLabellingModel

Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [4]:
recommender = DeepContentRecommender(None,None,None,'Recommender/DeepContent/use5-bestcheckpoint-0.08- 0.35.hdf5','dataset/song_preview/','dataset/deep_learning/label_map(boolean_mood).csv')
result = recommender.analyzeNewSong('/home/capt4ce/projects/major_project/dataset/song_preview/TRAAAAW128F429D538-mzm.jmksdiul.aac.p.m4a')
result.shape

# lModel = SongLabellingModel('Recommender/DeepContent/use5-bestcheckpoint-0.08- 0.35.hdf5','dataset/song_preview/','dataset/deep_learning/label_map(boolean_mood).csv')
# model = lModel.getModel()
# model

(1, 50)

In [5]:
result

array([[  1.14635390e-09,   2.18802157e-10,   2.91894148e-10,
          1.17291563e-07,   1.99895621e-11,   3.87927662e-10,
          1.17285524e-07,   1.30511268e-10,   8.23791382e-12,
          6.46717541e-11,   1.71904869e-11,   1.67403119e-10,
          4.77288174e-11,   1.10012815e-11,   9.26214834e-13,
          2.25955324e-10,   5.12563568e-11,   5.28332267e-10,
          2.93357033e-10,   1.91531643e-12,   2.74821507e-12,
          2.06678793e-12,   1.87133815e-11,   2.12216703e-11,
          2.43490905e-12,   3.27562533e-10,   3.13397364e-11,
          5.45534451e-12,   5.56423614e-12,   2.90404072e-11,
          1.35333422e-13,   2.96428193e-09,   1.82051822e-11,
          7.43556692e-12,   1.89356213e-12,   6.48672468e-13,
          1.42464881e-12,   1.64578840e-13,   2.43430304e-10,
          1.05204873e-11,   8.80749900e-12,   3.47953541e-11,
          1.22709495e-10,   1.52833113e-09,   5.08819542e-10,
          1.39932240e-07,   8.51788187e-08,   1.81001695e-07,
        